In [4]:
import copy as cp

In [5]:
class Game:    
    
    def __init__(self):
        self.clstr = Cluster()
        self.n = -1;
        print("[+] Game Created, please load a game board to proceed...")
        
    def to_move(self, s):
        self.test_method_failure()
        p = 1 #player
        return p
    
    def terminal_test(self, s):
        self.test_method_failure()
        return True
    
    def utility(self, s, p):
        self.test_method_failure()
        payoff = 1
        return payoff
    
    def actions(self, s):
        '''
            This is only returning empty spaces, available actions should not allow for KO rule breakers
            and maybe even stupid moves
        '''
        self.test_method_failure()
        return [(self.to_move(s), i, j) for i in range(self.n) for j in range(self.n) if go.board[i][j] == 0] 
    
    def result(self, s, a):
        self.test_method_failure()
        state = s
        return state
    
    def load_board(self, s):
        '''
            Loads the board from an opened file stream and saves it to the class field 'board'
            as weel as board dimension 'n', and first player 'p'
            this is done while checking for file integrity
        '''
        print("\t[+] Loading Board...")
        contents = s.read().split()
        try:
            self.n = int(contents[0]) # get board dimension
            self.p = int(contents[1]) # get first player to move
            self.board = [list(map(int, lst)) for lst in contents[2:] # load board into 2D list
                          if len(lst)==self.n or                      # check if each row has the correct number of elements
                          any(c in lst for c in ['3', '4', '5', '6', '7', '8', '9'])] #check if any digits above 2 in each row
        except ValueError:
            # if at any point a non numeric value is found failure is issued
            self.load_failure()
            return
        
        if len(self.board) == self.n:
            print("\t[+] Game Board Successfuly Loaded!")
        else:
            # if dimensions dont match return failure
            self.load_failure()
    
    def load_failure(self):
        print("\t\t[-] Bad File...")
        print("\t\t[-] Please try another file...")
        self.board = None
    
    def test_method_failure(self):
        if not self.board:
            print('\t[-] Game not initiated, please lad a game borad...')
            raise Exception('Error: Game not initiated.')

In [ ]:
class Grouper:
    
    def __init__(self):
        self.clusters = {1: {}, 2: {}}
        self.emptyPoints = set()
        self.n = -1
    
    def cluster_maker(self, p, board, row, column, key):
        if(row<0 or column<0 or row>=self.n or column>=self.n):
            return 
        
        if(board[row][column] == 0):
            self.clusters[p][key]["liberties"].update([(row, column)])
            return 
        
        if(board[row][column] != p):
            return 
        
        board[row][column] = -1
        self.clusters[p][key]["pieces"].update([(row, column)])

        self.cluster_maker(p, board, row-1, column, key)
        self.cluster_maker(p, board, row+1, column, key)
        self.cluster_maker(p, board, row, column-1, key)
        self.cluster_maker(p, board, row, column+1, key)
            
        
    def load_board(self, board, n):
        matrix = cp.deepcopy(board)
        self.n = n
        
        for i in range(n):
            for j in range(n):
                if(matrix[i][j] == 1):
                    key = len(self.clusters[1])+1
                    self.clusters[1].update({key: {"pieces": set(), "liberties": set()}})
                    self.cluster_maker(1, matrix, i, j, key)
                    
                elif(matrix[i][j] == 2):
                    key = len(self.clusters[2])+1
                    self.clusters[2].update({key: {"pieces": set(), "liberties": set()}})
                    self.cluster_maker(2, matrix, i, j, key)
                    
                elif(board[i][j] != -1):
                    self.emptyPoints.update([(i,j)])
    
    def get_liberties(self, row, column):
        liberties = []
        
        if(row-1>=0):
            if((row-1,column) in self.emptyPoints):
                liberties.append((row-1,column))
                
        if(row+1<self.n):
            if((row+1,column) in self.emptyPoints):
                liberties.append((row+1,column))
        
        if(column-1>=0): 
            if((row,column-1) in self.emptyPoints):
                liberties.append((row,column-1))
            
        if(column+1<self.n):
            if((row,column+1) in self.emptyPoints):
                liberties.append((row,column+1))
                
        return liberties
        
    def join_clusters(self, cluster1, cluster2):
        return {"pieces" : cluster1["pieces"].union(cluster2["pieces"]), "liberties" : cluster1["liberties"].union(cluster2["liberties"])}
    
    def add_2_cluster(self, p, key, row, column):
        liberties = self.get_liberties(row, column) 
        
        try:
            self.clusters[p][key]["liberties"].remove((row,column))
        except:
            pass
                
        self.clusters[p][key]["pieces"].update([(row,column)])
        self.clusters[p][key]["liberties"].update(liberties)
        
    def add_piece(self, p, row, column, state):          
        join_clstrs = []
        
        if(p==1):
            op = 2 
        else: 
            op = 1           
             
        for cluster in state.clusters[p].items():
            if((row,column) in cluster[1]["liberties"]):
                join_clstrs.append(cluster[0])
                
        if(len(join_clstrs) > 1):
            for i in join_clstrs[1:]:
                state.clusters[p][join_clstrs[0]] = self.join_clusters(state.clusters[p][join_clstrs[0]], state.clusters[p][i])
                del state.clusters[p][i]
                
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        elif(len(join_clstrs) == 1):
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        else:
            self.clusters[p].update({len(self.clusters[p])+1: {"pieces": set([(row,column)]), "liberties": set(self.get_liberties(row, column))}})
            
        for cluster in self.clusters[op].items():
            try:
                cluster[1]["liberties"].remove((row,column)) 
            except:
                pass
        self.emptyPoints.remove((row,column))

In [26]:
class Cluster:
    
    def __init__(self):
        self.clusters = {1: {}, 2: {}}
        self.emptyPoints = set()
        self.board = None
    
    def cluster_maker(self, p, matrix, n, row, column, key):
        if(row<0 or column<0 or row>=n or column>=n):
            return 
        
        if(matrix[row][column] == 0):
            self.clusters[p][key]["liberties"].update([(row, column)])
            return 
        
        if(matrix[row][column] != p):
            return 
        
        matrix[row][column] = -1
        self.clusters[p][key]["pieces"].update([(row, column)])
        print(str((row,column)))
        self.cluster_maker(p, matrix, n, row-1, column, key)
        self.cluster_maker(p, matrix, n, row+1, column, key)
        self.cluster_maker(p, matrix, n, row, column-1, key)
        self.cluster_maker(p, matrix, n, row, column+1, key)
            
        
    def load_board(self, matrix, n):
        self.board = cp.deepcopy(matrix)
        self.n = n
        
        for i in range(n):
            for j in range(n):
                if(matrix[i][j] == 1):
                    key = len(self.clusters[1])+1
                    self.clusters[1].update({key: {"pieces": set(), "liberties": set()}})
                    self.cluster_maker(1, matrix, n, i, j, key)
                elif(matrix[i][j] == 2):
                    key = len(self.clusters[2])+1
                    self.clusters[2].update({key: {"pieces": set(), "liberties": set()}})
                    self.cluster_maker(2, matrix, n, i, j, key)
                elif(matrix[i][j] != -1):
                    self.emptyPoints.update([(i,j)])
    
    def get_liberties(self, row, column):
        liberties = []
        
        if((row-1>=0) and ((row-1,column) in self.emptyPoints)):
            print(str((row-1,column)))
            liberties.append((row-1,column))
                
        if((row+1<self.n) and ((row+1,column) in self.emptyPoints)):
            print(str((row+1,column)))
            liberties.append((row+1,column))
        
        if((column-1>=0) and ((row,column-1) in self.emptyPoints)):
            print(str((row,column-1)))
            liberties.append((row,column-1))
            
        if((column+1<self.n) and ((row,column+1) in self.emptyPoints)):
            print(str((row,column+1)))
            liberties.append((row,column+1))
                
        return liberties
        
    def join_clusters(self, cluster1, cluster2):
        return {"pieces" : cluster1["pieces"].union(cluster2["pieces"]), "liberties" : cluster1["liberties"].union(cluster2["liberties"])}
    
    def add_2_cluster(self, p, key, row, column):
        liberties = self.get_liberties(row, column) 
        
        try:
            self.clusters[p][key]["liberties"].remove((row,column))
        except:
            pass
                
        self.clusters[p][key]["pieces"].update([(row,column)])
        self.clusters[p][key]["liberties"].update(liberties)
        
    def add_piece(self, p, row, column):
        if(self.board[row][column] != 0):
            raise Exception('Point already taken by player ' + str(self.board[row][column]))
            
        join_clstrs = []
        
        if(p==1):
            op = 2 
        else: 
            op = 1           
             
        for cluster in self.clusters[p].items():
            if((row,column) in cluster[1]["liberties"]):
                join_clstrs.append(cluster[0])
                
        if(len(join_clstrs) > 1):
            for i in join_clstrs[1:]:
                 self.clusters[p][join_clstrs[0]]= self.join_clusters(self.clusters[p][join_clstrs[0]], self.clusters[p][i])
                del self.clusters[p][i]
                
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        elif(len(join_clstrs) == 1):
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        else:
            self.clusters[p].update({len(self.clusters[p])+1: {"pieces": set([(row,column)]), "liberties": set(self.get_liberties(row, column))}})
            
        for cluster in self.clusters[op].items():
            try:
                cluster[1]["liberties"].remove((row,column)) 
            except:
                pass
        self.emptyPoints.remove((row,column))

In [27]:
go = Game()
go.load_board(open('../boards/board_2.txt', 'r'))
matrix = cp.deepcopy(go.board)
n = go.n
matrix

[+] Game Created, please load a game board to proceed...
	[+] Loading Board...
	[+] Game Board Successfuly Loaded!


[[0, 0, 1, 1], [2, 2, 2, 1], [0, 2, 0, 0], [0, 0, 0, 0]]

In [28]:
clstr = Cluster()
clstr.load_board(matrix, n)
row = 0
column = 1

(0, 2)
(0, 3)
(1, 3)
(1, 0)
(1, 1)
(2, 1)
(1, 2)


In [45]:
clstr.add_piece(1, 3, 3)

In [46]:
clstr.emptyPoints

set()

In [47]:
smallP = float('inf')
for cls in clstr.clusters[1].values():
    newcls = len(cls['liberties'])
    if newcls < smallP:
        smallP = newcls

In [48]:
smallP

0